# Clasificación multiclase

## Introduction

En este ejercicio se implementa la regresion one-vs-all y una red neuronal para reconocimiento de digitos.
Dataset:https://www.kaggle.com/datasets/colewelkins/cardiovascular-disease

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [35]:
import pandas as pd
import numpy as np

# 4 etiquetas, de 0 a 3 (normal = 0, hypertension1 = 1, hypertension2 = 2, Elevated: 3)
num_labels = 4

# Cargar los datos usando Pandas
data = pd.read_csv('/content/drive/MyDrive/datasets/cardio_data_processed.csv')

# Omitir la columna 0(no tiene relevancia pq es la numeración de las filas) en X y la columna 17(es una columna repetida de Y)
X_test = data.iloc[0:54500, 1:15].values
y_test = data.iloc[0:54500:, 16].values

X_train = data.iloc[54500:, 1:15].values
y_train = data.iloc[54500:, 16].values


X=X_test
y=y_test

#X=X_train
#y=y_train

# Mapear los valores de la columna 16 según la lógica (normal = 0, hypertension1 = 1, hypertension2 = 2, Elevated: 3)
mapping = {
    'Hypertension Stage 1': 1,
    'Hypertension Stage 2': 2,
    'Normal': 0,
    'Elevated': 3
}

# Reemplazar los valores en la columna de etiquetas según el diccionario de mapeo
y_mapped = np.array([mapping[val] for val in y])
y=y_mapped
y[y == 4] = 0



print(X)

m = y.size

[[1.83930000e+04 2.00000000e+00 1.68000000e+02 ... 0.00000000e+00
  5.00000000e+01 2.19671202e+01]
 [2.02280000e+04 1.00000000e+00 1.56000000e+02 ... 1.00000000e+00
  5.50000000e+01 3.49276792e+01]
 [1.88570000e+04 1.00000000e+00 1.65000000e+02 ... 1.00000000e+00
  5.10000000e+01 2.35078053e+01]
 ...
 [1.95540000e+04 1.00000000e+00 1.60000000e+02 ... 1.00000000e+00
  5.30000000e+01 3.39843750e+01]
 [1.89110000e+04 1.00000000e+00 1.53000000e+02 ... 1.00000000e+00
  5.10000000e+01 3.11845871e+01]
 [2.05350000e+04 1.00000000e+00 1.65000000e+02 ... 0.00000000e+00
  5.60000000e+01 2.75482094e+01]]


In [36]:
print(X)
print(y)

[[1.83930000e+04 2.00000000e+00 1.68000000e+02 ... 0.00000000e+00
  5.00000000e+01 2.19671202e+01]
 [2.02280000e+04 1.00000000e+00 1.56000000e+02 ... 1.00000000e+00
  5.50000000e+01 3.49276792e+01]
 [1.88570000e+04 1.00000000e+00 1.65000000e+02 ... 1.00000000e+00
  5.10000000e+01 2.35078053e+01]
 ...
 [1.95540000e+04 1.00000000e+00 1.60000000e+02 ... 1.00000000e+00
  5.30000000e+01 3.39843750e+01]
 [1.89110000e+04 1.00000000e+00 1.53000000e+02 ... 1.00000000e+00
  5.10000000e+01 3.11845871e+01]
 [2.05350000e+04 1.00000000e+00 1.65000000e+02 ... 0.00000000e+00
  5.60000000e+01 2.75482094e+01]]
[1 2 1 ... 2 1 1]


In [37]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [38]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [39]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
# X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

In [40]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
rand_indices = np.random.choice(m, 100, replace=False)
sel = X[rand_indices, :]

# displayData(sel)

In [41]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [42]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))# factor de regularizacion (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [43]:
def oneVsAll(X, y, num_labels, lambda_):
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [44]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(4, 15)


In [45]:
print(all_theta)

[[-1.26218101e+01 -3.80941399e-02 -9.91916924e-02 -9.01719683e-02
   3.18799362e-02 -8.39090987e+00 -4.69723866e+00  1.15729548e-01
  -4.72077654e-02 -1.40696353e-02  7.84790924e-03 -7.85849270e-03
  -3.44233599e-02 -7.87338663e-02 -2.93515848e-02]
 [ 3.45243319e-01  1.39911527e-01  1.89508009e-02  1.86282102e-01
  -7.78909617e-02 -6.97431297e-01  4.04202806e-01 -5.47149101e-02
   2.54813143e-02 -2.13724673e-02 -2.62531274e-02 -2.06861588e-02
  -1.00021195e-01  4.51516152e-02  1.15792595e-01]
 [-3.20542165e+00 -2.13312272e-02 -5.43649006e-04  5.25082401e-02
  -1.20657443e-01  1.80245379e+00  2.93425533e+00 -4.62802450e-02
   2.49770731e-02 -2.98156260e-02  3.86422410e-04 -9.22946482e-03
   5.86024885e-02 -1.54334762e-02  3.86425268e-02]
 [-4.21032571e+00  2.93229379e-02  7.12184525e-02  1.90799600e-01
  -1.43770360e-01  1.00319256e+00 -2.17445955e+00 -1.76302427e-01
   3.91727528e-02 -3.77281007e-02  3.43337931e-02 -3.44900202e-02
  -2.39380470e-01  5.23853144e-02  1.08217630e-01]]


In [46]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [47]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[1:140, :].copy()
print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
XPrueba = np.concatenate([np.ones((139, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

# displayData(X[1002:1003, :])
print(y[1:140])

(54500, 14)
Precision del conjuto de entrenamiento: 89.86%
(139, 14)
(139, 15)
[2 1 2 0 1 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 1 2 0 2 2 1 1 1 1 1 2 0 1 2 1 1 1
 1 2 0 1 2 0 2 2 2 1 1 1 0 2 2 2 1 1 0 1 2 2 0 2 1 1 1 1 1 2 2 0 1 2 0 2 2
 1 1 2 2 1 1 1 1 2 1 0 2 2 1 1 2 2 2 2 1 0 0 2 1 1 0 0 1 1 2 1 0 1 0 1 1 0
 1 1 2 1 1 0 0 1 1 2 0 2 1 1 2 1 2 1 1 2 1 1 1 1 1 1 1 1]
[2 1 2 0 1 1 1 0 0 1 1 1 0 1 1 1 0 0 3 1 1 1 0 2 1 1 1 1 1 1 2 0 1 2 1 1 1
 1 2 0 1 2 3 1 2 2 1 1 3 0 2 2 1 1 2 0 1 2 2 0 2 1 1 1 1 1 2 2 0 1 2 0 2 2
 1 1 2 2 1 3 3 1 2 1 0 2 2 1 1 2 2 2 2 1 0 0 2 1 1 0 0 1 1 2 1 0 1 0 1 1 0
 3 1 2 1 1 0 0 1 1 2 0 2 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 2]


In [48]:
#usamos x_train para comprobar que el aprendizaje predice los valores de y_train
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X_train[1:140, :].copy()
print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
# Normalizar XPrueba utilizando las medias y desviaciones estándar de X_train
XPrueba_norm = (XPrueba - mu) / sigma

# Agregar la columna de unos
XPrueba_norm_con_intercepto = np.concatenate([np.ones((XPrueba_norm.shape[0], 1)), XPrueba_norm], axis=1)

# Realizar predicciones en el conjunto de prueba normalizado
p = np.argmax(sigmoid(XPrueba_norm_con_intercepto.dot(all_theta.T)), axis=1)
print(p)

# displayData(X[1002:1003, :])
print(y_train[1:140])

Precision del conjuto de entrenamiento: 89.86%
(139, 14)
[1 2 1 1 0 1 2 1 1 1 2 0 2 2 1 2 1 1 1 2 1 1 1 1 1 1 0 2 1 1 1 2 1 1 2 1 1
 1 1 1 1 0 2 2 1 2 0 1 0 1 1 1 1 2 2 1 1 1 1 1 1 2 0 1 2 0 0 2 1 2 1 1 1 0
 1 1 0 1 2 2 1 0 2 1 1 1 1 2 3 0 1 1 2 2 1 1 1 2 1 2 2 1 1 1 1 1 1 1 0 1 1
 1 0 0 0 0 0 1 1 1 1 2 1 1 1 1 1 1 2 2 1 1 1 0 1 1 1 1 1]
['Hypertension Stage 1' 'Hypertension Stage 2' 'Hypertension Stage 1'
 'Hypertension Stage 1' 'Normal' 'Hypertension Stage 1'
 'Hypertension Stage 2' 'Hypertension Stage 1' 'Hypertension Stage 1'
 'Hypertension Stage 1' 'Hypertension Stage 2' 'Normal'
 'Hypertension Stage 2' 'Hypertension Stage 2' 'Hypertension Stage 1'
 'Hypertension Stage 2' 'Hypertension Stage 1' 'Hypertension Stage 1'
 'Elevated' 'Hypertension Stage 2' 'Hypertension Stage 1'
 'Hypertension Stage 1' 'Hypertension Stage 1' 'Hypertension Stage 1'
 'Hypertension Stage 1' 'Hypertension Stage 1' 'Normal'
 'Hypertension Stage 2' 'Hypertension Stage 1' 'Hypertension Stage 2'
 'Hypertension